In [2]:
from eegdash import EEGDash

In [3]:
eegdash = EEGDash()
eegdash.find({'task': 'FaceRecognition'})

Pinged your deployment. You successfully connected to MongoDB!
Found 18 records


TypeError: SignalstoreBIDS.get() missing 1 required positional argument: 'query'